In [4]:
import pandas as pd
import numpy as np


df = pd.DataFrame(np.random.randint(5,size=(3,4)),columns=('A','B','C','D'))
df.iloc[1,2]=None
df.iloc[2,3]=np.nan

df

,A,B,C,D
0,3,0,1.0,2.0
1,0,2,NaN,3.0
2,1,4,3.0,NaN


In [10]:
df.isnull().sum(axis=0)

A    0
B    0
C    1
D    1
dtype: int64

In [17]:
df.dropna(how='all',axis=1)

,A,B,C,D
0,3,0,1.0,2.0
1,0,2,NaN,3.0
2,1,4,3.0,NaN


결측데이터보정

In [22]:
from sklearn.preprocessing import Imputer
imr = Imputer(missing_values='NaN',strategy='mean',axis=0)
imr = imr.fit(df)
imputed_data=imr.transform(df.values)
imputed_data

array([[3. , 0. , 1. , 2. ],
       [0. , 2. , 2. , 3. ],
       [1. , 4. , 3. , 2.5]])

In [27]:
# 텍스트 결측값의 보정
df = pd.DataFrame(np.random.randint(5, size=(3, 4)), columns=['A', 'B', 'C', 'D'])
df.iloc[1, 2] = None
df.iloc[2, 3] = np.nan

import numpy as np
df1 = df.replace(np.nan, 'text')
df1 = df1.astype(str)

df1

,A,B,C,D
0,1,3,2.0,3.0
1,4,3,text,2.0
2,4,0,2.0,text


One-Hot Encoding 실습

In [29]:
import pandas as pd
import numpy.random as np

np.seed(111)

# 연습용 데이터셋 생성 함수
def CreateDataSet(Number=1):
    
    Output = []
    
    for i in range(Number):
        rng = pd.date_range(start='1/1/2009', end='12/31/2012', freq='W-MON')
        data = np.randint(low=25,high=1000,size=len(rng))
        states = ['GA','FL','fl','NY','NJ','TX']
        random_states = [states[np.randint(low=0,high=len(states))] for i in range(len(rng))]    
        Output.extend(zip(random_states, data, rng))
        
    return Output

In [30]:
# 연습용 데이터셋 생성
dataset = CreateDataSet(4)
df = pd.DataFrame(data=dataset, columns=['State','CustomerCount','StatusDate'])
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836 entries, 0 to 835
Data columns (total 3 columns):
State            836 non-null object
CustomerCount    836 non-null int64
StatusDate       836 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 19.7+ KB


,State,CustomerCount,StatusDate
0,GA,877,2009-01-05
1,NJ,901,2009-01-12
2,fl,749,2009-01-19
3,fl,111,2009-01-26
4,GA,300,2009-02-02


In [31]:
# Dataframe feature 전처리
df['State'] = df.State.apply(lambda x: x.upper())

In [33]:
# State 변수 더미변수로 만들기, One-Hot Encoding 완성
df_one_hot_encoded = pd.get_dummies(df.State) ##get_dummies 함수
type(df_one_hot_encoded)
result = pd.concat([df, df_one_hot_encoded], axis=1)
result.head()

,State,CustomerCount,StatusDate,FL,GA,NJ,NY,TX
0,GA,877,2009-01-05,0,1,0,0,0
1,NJ,901,2009-01-12,0,0,1,0,0
2,FL,749,2009-01-19,1,0,0,0,0
3,FL,111,2009-01-26,1,0,0,0,0
4,GA,300,2009-02-02,0,1,0,0,0


In [41]:
result=result.drop('State',1)

KeyError: "labels ['State'] not contained in axis"

In [42]:
result.head()

,CustomerCount,StatusDate,FL,GA,NJ,NY,TX
0,877,2009-01-05,0,1,0,0,0
1,901,2009-01-12,0,0,1,0,0
2,749,2009-01-19,1,0,0,0,0
3,111,2009-01-26,1,0,0,0,0
4,300,2009-02-02,0,1,0,0,0


In [44]:
# one-hot encoding 예제
import keras
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation

Using TensorFlow backend.


In [45]:
# 훈련셋과 시험셋 로딩
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# 데이터셋 전처리
X_train = X_train.reshape(60000, 784).astype('float32') / 255.0
X_test = X_test.reshape(10000, 784).astype('float32') / 255.0

# one-hot encoding
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)

11493376/11490434 [==============================] - 16s 1us/step


In [46]:
# 모델링 구성
model = Sequential()
model.add(Dense(units=64, input_dim=28*28, activation='relu'))
model.add(Dense(units=10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

hist = model.fit(X_train, Y_train, epochs=3, batch_size=32)

Epoch 1/3
60000/60000 [==============================] - 3s 48us/step - loss: 0.7133 - acc: 0.8150
Epoch 2/3
60000/60000 [==============================] - 3s 47us/step - loss: 0.3482 - acc: 0.9023
Epoch 3/3
60000/60000 [==============================] - 3s 47us/step - loss: 0.2987 - acc: 0.9152


In [47]:
# 샘플 테스트
from numpy import argmax

xhat_idx = np.random.choice(X_test.shape[0], 5)
xhat = X_test[xhat_idx]
yhat = model.predict_classes(xhat)

for i in range(5):
    print('True : ' + str(argmax(Y_test[xhat_idx[i]])) + ', Predict : ' + str(yhat[i]))
    

## 참고용 실행 코드
argmax(Y_test[xhat_idx[0]])
Y_test[xhat_idx[0]]

AttributeError: 'builtin_function_or_method' object has no attribute 'choice'